In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW  
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, DistilBertForSequenceClassification, get_scheduler
import pandas as pd
import numpy as np
from tqdm import tqdm

2025-04-23 10:08:42.590710: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745402922.919039      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745402923.007243      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
from transformers import AutoModel

In [15]:
df = pd.read_csv("/kaggle/input/cleaned-arabic-text-svc-dataset-csv/cleaned_arabic_text_svc_dataset.csv")

# 2. Make sure 'label' column is already binary (0 = Human, 1 = AI)
print(df['label'].value_counts())  # Optional: view label distribution

# 3. Split dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(), df['label'].tolist(), test_size=0.2, stratify=df['label'], random_state=42
)

# 4. Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("asafaya/bert-base-arabic")
model = AutoModelForSequenceClassification.from_pretrained("asafaya/bert-base-arabic", num_labels=2)

# 5. Dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# 6. Create DataLoaders
train_dataset = TextDataset(train_texts, train_labels, tokenizer)
val_dataset = TextDataset(val_texts, val_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)

# 7. Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = len(train_loader) * 3  # 3 epochs
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# 8. Train the model
model.train()
for epoch in range(3):
    print(f"Epoch {epoch+1}")
    for batch in tqdm(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

# 9. Save model & tokenizer
model.save_pretrained("./arabic_model_v2")
tokenizer.save_pretrained("./arabic_model_v2")

label
0    5000
1    5000
Name: count, dtype: int64


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at asafaya/bert-base-arabic and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1


100%|██████████| 1000/1000 [07:06<00:00,  2.34it/s]


Epoch 2


100%|██████████| 1000/1000 [07:06<00:00,  2.34it/s]


Epoch 3


100%|██████████| 1000/1000 [07:06<00:00,  2.35it/s]


('./arabic_model_v2/tokenizer_config.json',
 './arabic_model_v2/special_tokens_map.json',
 './arabic_model_v2/vocab.txt',
 './arabic_model_v2/added_tokens.json',
 './arabic_model_v2/tokenizer.json')

In [16]:
from sklearn.metrics import classification_report

# Switch to eval mode
model.eval()

all_preds = []
all_labels = []

# Disable gradient calculation for evaluation
with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Print classification report
report = classification_report(all_labels, all_preds, target_names=["Human", "AI"])
print(report)


              precision    recall  f1-score   support

       Human       1.00      1.00      1.00      1000
          AI       1.00      1.00      1.00      1000

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000



In [17]:
import shutil

shutil.make_archive("arabic_model_v2", 'zip', "./arabic_model_v2")


'/kaggle/working/arabic_model_v2.zip'

In [19]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F

# 1. Load saved model and tokenizer
model_path = "./arabic_model_v2"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()

# 2. Move model to correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 3. Function to predict single sample
def predict_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = F.softmax(logits, dim=1)
        prediction = torch.argmax(probs, dim=1).item()
        confidence = probs[0][prediction].item()

    label_map = {0: "Human", 1: "AI"}
    return label_map[prediction], confidence

In [20]:
text_sample = "شعر سامر بالفضول، فتوجه إلى المدرسة القديمة، ووجد شجرة تين ضخمة. بدأ بالحفر تحتها، وهناك وجد صندوقًا صغيرًا يحتوي على صور قديمة ورسائل حب بين زوجين عاشا في ذلك الحي منذ أكثر من خمسين سنة. كانت القصة التي كُتبت في تلك الرسائل أجمل من أي كنز مادي."
label, confidence = predict_text(text_sample)
print(f"Prediction: {label}, Confidence: {confidence:.2%}")

Prediction: AI, Confidence: 100.00%


In [21]:
text_sample = "لذكاء الاصطناعي (AI) هو مجال من مجالات علوم الكمبيوتر يهدف إلى إنشاء أنظمة قادرة على محاكاة الذكاء البشري. يشمل الذكاء الاصطناعي تطوير الخوارزميات والنماذج التي تمكّن الآلات من أداء مهام تتطلب عادة ذكاء بشري مثل التعلم، والتفكير، واتخاذ القرارات، والتفاعل مع البيئة."
label, confidence = predict_text(text_sample)
print(f"Prediction: {label}, Confidence: {confidence:.2%}")

Prediction: AI, Confidence: 98.07%


In [23]:
text_sample = "فـــالكمبیوتر یمكـــن أن یطـــور الفكـــرة الواحـــدة بـــآلاف الأفكـــار، إلـــى جانـــب قدرتـــه علـــى التخـزین وتقـدیم البـدائل وعمـل صـیاغات لانهائیـة یمكـن الاسـتفادة منهـا فـي بنـاء العمـل الفنـي الواحـد بـل والتنـوع اللانهـائي لـه مـن حیـث اسـتخدام الحـذف والإضـافة ، وتعتبـر الألـوان وكـذا الـدرجات اللونیـة والخطـوط وملامـس الأشـكال والأرضـیات والمونتـاج بـین عناصـر العمـل الفنـي ووضعها في أطر بصریة متعددة بالإضـافة إلـى الدقـة الـشدیدة فـي صـیاغة الأشـكال بـل الأكثـر من ذلك یمكن استخدام الذاكرة في استرجاع أعمال سبق تخزینهـا واختیـار بعـض الأشـكال منهـا فـــى إنـــشاء عمـــل جدیـــد بالإضـــافة لإمكانیـــة طبـــع كـــل هـــذه التكونیـــات والمقارنـــة فیمـــا بینهـــا والاحتفاظ بها مسجلة لإعادة طباعتها عند الحاجة"
label, confidence = predict_text(text_sample)
print(f"Prediction: {label}, Confidence: {confidence:.2%}")

Prediction: Human, Confidence: 100.00%


In [27]:
text_sample =  "فـــالكمبیوتر یمكـــن أن یطـــور الفكـــرة الواحـــدة بـــآلاف الأفكـــار، إلـــى جانـــب قدرتـــه علـــى التخـزین وتقـدیم البـدائل وعمـل صـیاغات لانهائیـة یمكـن الاسـتفادة منهـا فـي بنـاء العمـل الفنـي الواحـد بـل والتنـوع اللانهـائي لـه مـن حیـث اسـتخدام الحـذف والإضـافة ، وتعتبـر الألـوان وكـذا الـدرجات اللونیـة والخطـوط وملامـس الأشـكال والأرضـیات والمونتـاج بـین عناصـر العمـل الفنـي ووضعها في أطر بصریة متعددة بالإضـافة إلـى الدقـة الـشدیدة فـي صـیاغة الأشـكال بـل الأكثـر من ذلك یمكن استخدام الذاكرة في استرجاع أعمال سبق تخزینهـا واختیـار بعـض الأشـكال منهـا فـــى إنـــشاء عمـــل جدیـــد بالإضـــافة لإمكانیـــة طبـــع كـــل هـــذه التكونیـــات والمقارنـــة فیمـــا بینهـــا والاحتفاظ بها مسجلة لإعادة طباعتها عند الحاجة"
label, confidence = predict_text(text_sample)
print(f"Prediction: {label}, Confidence: {confidence:.2%}")

Prediction: Human, Confidence: 100.00%


In [28]:
text_sample = "يهدف الذكاء الاصطناعي إلى جعل الآلات أكثر قدرة على التفكير والتعلم مثل البشر، ويستخدم في العديد من التطبيقات مثل السيارات الذاتية القيادة، المساعدات الرقمية، الترجمة التلقائية، وأدوات كشف النصوص المولدة بالذكاء الاصطناعي."
label, confidence = predict_text(text_sample)
print(f"Prediction: {label}, Confidence: {confidence:.2%}")

Prediction: AI, Confidence: 99.92%
